In [0]:
# Importanto a biblioteca

import pyspark.pandas as ps

In [0]:
# Lendo os arquivos Parquet

df_junto = ps.read_parquet("dbfs:/databricks-results/bronze/*/*/*")
df_junto.head()

,moeda,taxa,data
0,SCR,2.751143,2023-04-01
1,SDG,117.907263,2023-04-01
2,SEK,2.049749,2023-04-01
3,SGD,0.262833,2023-04-01
4,SHP,0.240308,2023-04-01


In [0]:
# Filtrando as moedas:

moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto[df_junto['moeda'].isin(moedas)]
df_moedas.head()

,moeda,taxa,data
24,USD,0.197499,2023-04-01
68,USD,0.197449,2023-04-03
112,USD,0.197153,2023-04-04
156,USD,0.198622,2023-04-05
200,USD,0.197765,2023-04-07


In [0]:
# Convertendo a coluna para datetime

df_moedas['data'] = ps.to_datetime(df_moedas['data'])

In [0]:
# Pivotando o DataFrame

resultado_taxas_conversao = df_moedas.pivot_table(
    values='taxa', 
    index=['data'], 
    columns='moeda', 
    aggfunc='first'
    ).sort_index(ascending=False)

resultado_taxas_conversao.head()

moeda,EUR,GBP,USD
data,,,
2023-06-03,0.188000,0.161948,0.201641
2023-06-02,0.188068,0.161989,0.201693
2023-05-31,0.184947,0.158901,0.197773
2023-05-30,0.185027,0.159981,0.198558
2023-05-29,0.186059,0.161327,0.199219


In [0]:
resultado_valores_reais = resultado_taxas_conversao.copy()

In [0]:
for moeda in moedas:
    resultado_valores_reais[moeda] = resultado_valores_reais[moeda]\
            .apply(
                lambda x: round(1 / x, 4)
            )


In [0]:
resultado_valores_reais.head()

moeda,EUR,GBP,USD
data,,,
2023-04-01,5.5042,6.2507,5.0633
2023-04-02,5.4718,6.2230,5.0638
2023-04-03,5.5257,6.2915,5.0646
2023-04-04,5.5583,6.3391,5.0722
2023-04-05,5.4899,6.2745,5.0347
